In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
model_id = "naver/splade-cocondenser-ensembledistil"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

In [ ]:
text = "I love Python very much because it is a very interesting computer language"

tokens = tokenizer(text, return_tensors="pt")
output = model(**tokens)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -6.1202,  -8.3422,  -7.6260,  ...,  -7.6751,  -7.9525,  -6.1075],
         [ -6.7719,  -8.9798,  -7.8527,  ...,  -8.3719,  -8.6289,  -6.6373],
         [ -9.3684, -10.5532,  -9.2008,  ...,  -9.6652,  -9.8406,  -8.1002],
         ...,
         [ -6.8283,  -8.5560,  -8.3984,  ...,  -8.6114,  -8.3427,  -6.3242],
         [ -6.7843,  -8.5415,  -8.5728,  ...,  -8.4019,  -8.2313,  -6.7143],
         [-21.3627, -17.6899, -16.6674,  ..., -17.5429, -16.4934, -18.7595]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
output.logits.shape

torch.Size([1, 15, 30522])

In [ ]:
import torch

vec = torch.max(torch.log(1 + torch.relu(output.logits)) * tokens["attention_mask"].unsqueeze(-1), dim=1)[0].squeeze()
vec.shape

torch.Size([30522])

In [ ]:
vec

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)

In [ ]:
cols = vec.nonzero().squeeze().cpu().tolist()
print(len(cols))

58


In [ ]:
weights = vec[cols].cpu().tolist()
sparse_dict = dict(zip(cols, weights))
sparse_dict

{1045: 0.3497620224952698,
 1052: 0.45730113983154297,
 2009: 0.34368041157722473,
 2061: 0.07585734128952026,
 2138: 0.9671566486358643,
 2172: 0.2688765525817871,
 2190: 0.2166033685207367,
 2200: 0.8433794379234314,
 2204: 0.553397536277771,
 2293: 1.1777918338775635,
 2428: 0.31924358010292053,
 2653: 1.2211918830871582,
 2748: 0.355165958404541,
 2759: 0.2548351287841797,
 2978: 0.2851647138595581,
 3112: 0.28132161498069763,
 3213: 0.006266824435442686,
 3274: 1.1782046556472778,
 3342: 0.18472643196582794,
 3407: 0.3151716887950897,
 3835: 0.41453754901885986,
 3889: 0.21852819621562958,
 3894: 0.15836818516254425,
 4007: 0.5925108790397644,
 4037: 0.13534662127494812,
 4083: 0.40074968338012695,
 4132: 0.0072813136503100395,
 4155: 1.3275483846664429,
 4553: 0.2317194938659668,
 4569: 0.13223373889923096,
 4669: 0.2612866461277008,
 4730: 0.2721441388130188,
 4913: 0.15388791263103485,
 5186: 0.027692044153809547,
 5202: 0.11551856994628906,
 5440: 0.13425996899604797,
 5454: 0

In [ ]:
idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}

In [ ]:
sparse_dict_tokens = {idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)}

sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}

sparse_dict_tokens

{'python': 2.7,
 'interesting': 1.6,
 'languages': 1.33,
 'snake': 1.33,
 'language': 1.22,
 'love': 1.18,
 'computer': 1.18,
 'because': 0.97,
 'very': 0.84,
 'computers': 0.72,
 'pc': 0.7,
 'java': 0.68,
 'software': 0.59,
 'good': 0.55,
 'p': 0.46,
 'useful': 0.45,
 'nice': 0.41,
 'fuzzy': 0.41,
 'learning': 0.4,
 'yes': 0.36,
 'linux': 0.36,
 'i': 0.35,
 'it': 0.34,
 'really': 0.32,
 'happy': 0.32,
 'greg': 0.32,
 'bit': 0.29,
 'success': 0.28,
 'much': 0.27,
 'programming': 0.27,
 'liked': 0.26,
 'popular': 0.25,
 'learn': 0.23,
 'best': 0.22,
 'steve': 0.22,
 'robot': 0.21,
 'remember': 0.18,
 'magic': 0.16,
 'raven': 0.16,
 'dave': 0.15,
 'popularity': 0.15,
 'website': 0.14,
 'fun': 0.13,
 'favorite': 0.13,
 'font': 0.13,
 'dragon': 0.12,
 'excellent': 0.09,
 'so': 0.08,
 'choose': 0.05,
 'terry': 0.05,
 'perry': 0.05,
 'fascinating': 0.05,
 'jay': 0.04,
 'extremely': 0.03,
 'happiness': 0.02,
 'writer': 0.01,
 'platform': 0.01,
 'jeremy': 0.01}

In [ ]:
!pip install git+https://github.com/naver/splade.git
from splade.models.transformer_rep import Splade

  Cloning https://github.com/naver/splade.git to /tmp/pip-req-build-wbpth80s
  Running command git clone --filter=blob:none --quiet https://github.com/naver/splade.git /tmp/pip-req-build-wbpth80s
  Resolved https://github.com/naver/splade.git to commit 3781228d5f07e7a6ae14a479e469a715de79e976
  Preparing metadata (setup.py) ... done


In [ ]:
sparse_model_id = "naver/splade-cocondenser-ensembledistil"

sparse_model = Splade(sparse_model_id, agg="max")
sparse_model.eval()

In [ ]:
with torch.no_grad():
    sparse_emb = sparse_model(d_kwargs=tokens)["d_rep"].squeeze()

sparse_emb.shape

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


torch.Size([30522])

In [ ]:
texts = [
   "Programmed cell death (PCD) is the regulated death of cells within an organism",
   "How is the scheduled death of cells within a living thing regulated?",
   "Photosynthesis is the process of storing light energy as chemical energy in cells"
]

In [ ]:
tokens = tokenizer(
    texts,
    return_tensors="pt",
    padding=True,
    truncation=True
)

output = model(**tokens)

vecs = torch.max(torch.log(1 + torch.relu(output.logits)) * tokens["attention_mask"].unsqueeze(-1), dim=1)[0].squeeze().detach().cpu().numpy()
vecs.shape

(3, 30522)

In [ ]:
import numpy as np

sim = np.zeros((vecs.shape[0], vecs.shape[0]))

for i, vec in enumerate(vecs):
    sim[i,:] = np.dot(vec, vecs.T) / (np.linalg.norm(vec) * np.linalg.norm(vecs, axis=1))

sim

array([[0.99999988, 0.54609394, 0.20535833],
       [0.54609394, 1.        , 0.20411888],
       [0.20535833, 0.20411888, 1.00000012]])